In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import optuna


In [25]:
df = pd.read_csv('cleaned_dataset.csv')

In [26]:
print(df.shape)
print(df.info())
print(df.describe())

(94891, 30)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94891 entries, 0 to 94890
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bathroomcount      94891 non-null  int64  
 1   bedroomcount       94891 non-null  int64  
 2   constructionyear   94891 non-null  int64  
 3   country            94891 non-null  object 
 4   district           94891 non-null  object 
 5   fireplace          94891 non-null  int64  
 6   floodingzone       94891 non-null  object 
 7   furnished          94891 non-null  int64  
 8   garden             94891 non-null  int64  
 9   kitchen            94891 non-null  int64  
 10  livingarea         94891 non-null  float64
 11  locality           94888 non-null  object 
 12  monthlycharges     94891 non-null  float64
 13  numberoffacades    94891 non-null  int64  
 14  peb                94891 non-null  object 
 15  postalcode         94891 non-null  int64  
 16  price     

In [27]:
df.columns

Index(['bathroomcount', 'bedroomcount', 'constructionyear', 'country',
       'district', 'fireplace', 'floodingzone', 'furnished', 'garden',
       'kitchen', 'livingarea', 'locality', 'monthlycharges',
       'numberoffacades', 'peb', 'postalcode', 'price', 'propertyid',
       'province', 'region', 'roomcount', 'showercount', 'stateofbuilding',
       'subtypeofproperty', 'surfaceofplot', 'swimmingpool', 'terrace',
       'toiletcount', 'typeofproperty', 'typeofsale'],
      dtype='object')

In [28]:
df

,bathroomcount,bedroomcount,constructionyear,country,district,fireplace,floodingzone,furnished,garden,kitchen,...,roomcount,showercount,stateofbuilding,subtypeofproperty,surfaceofplot,swimmingpool,terrace,toiletcount,typeofproperty,typeofsale
0,1,1,1969,Belgium,Brugge,0,NON_FLOOD_ZONE,0,0,1,...,1,0,4,flat_studio,203,0,1,1,2,residential_sale
1,6,13,1920,Belgium,Tournai,0,NON_FLOOD_ZONE,0,0,2,...,31,1,4,apartment_block,130,0,0,5,1,residential_sale
2,2,4,2008,Belgium,Brugge,0,NON_FLOOD_ZONE,1,0,1,...,3,0,4,house,0,0,0,2,1,residential_sale
3,1,4,1979,Belgium,Veurne,0,NON_FLOOD_ZONE,0,1,1,...,9,1,2,house,170,0,1,2,1,residential_sale
4,0,2,1972,Belgium,Hasselt,0,NON_FLOOD_ZONE,0,0,1,...,1,0,5,apartment,400,0,1,1,2,residential_sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94886,1,1,2017,Belgium,Tongeren,0,NON_FLOOD_ZONE,0,0,2,...,5,0,4,service_flat,286,0,1,1,2,residential_sale
94887,1,3,2024,Belgium,Gent,0,NON_FLOOD_ZONE,0,1,1,...,4,1,4,house,234,0,0,0,1,residential_sale
94888,4,4,2020,Belgium,Antwerp,0,NON_FLOOD_ZONE,0,0,2,...,8,1,3,apartment_block,202,0,0,1,1,residential_sale
94889,1,2,2014,Belgium,Antwerp,0,NON_FLOOD_ZONE,0,0,3,...,8,1,4,apartment,606,0,1,1,2,residential_sale


refining cleaned data

In [29]:
def clean_data(df):
    # Drop column: 'country'
    df = df.drop(columns=['country'])
    # Drop column: 'fireplace'
    df = df.drop(columns=['fireplace'])
    # Drop column: 'monthlycharges'
    df = df.drop(columns=['monthlycharges'])
    # Drop column: 'locality'
    df = df.drop(columns=['locality'])
    # Drop column: 'propertyid'
    df = df.drop(columns=['propertyid'])
    # Drop column: 'constructionyear'
    df = df.drop(columns=['constructionyear'])
    df = df.drop(columns=['furnished'])
    df = df.drop(columns=['roomcount'])
    df = df.drop(columns=['postalcode'])
   
    return df


df_clean = clean_data(df.copy())
df_clean.head()


,bathroomcount,bedroomcount,district,floodingzone,garden,kitchen,livingarea,numberoffacades,peb,price,...,region,showercount,stateofbuilding,subtypeofproperty,surfaceofplot,swimmingpool,terrace,toiletcount,typeofproperty,typeofsale
0,1,1,Brugge,NON_FLOOD_ZONE,0,1,29.0,2,B,99000,...,Flanders,0,4,flat_studio,203,0,1,1,2,residential_sale
1,6,13,Tournai,NON_FLOOD_ZONE,0,2,391.0,3,D,765000,...,Wallonie,1,4,apartment_block,130,0,0,5,1,residential_sale
2,2,4,Brugge,NON_FLOOD_ZONE,0,1,111.0,2,B,399000,...,Flanders,0,4,house,0,0,0,2,1,residential_sale
3,1,4,Veurne,NON_FLOOD_ZONE,1,1,113.6,2,F,230000,...,Flanders,1,2,house,170,0,1,2,1,residential_sale
4,0,2,Hasselt,NON_FLOOD_ZONE,0,1,92.0,2,B,198000,...,Flanders,0,5,apartment,400,0,1,1,2,residential_sale


In [30]:
def clean_data(df_clean):
    # Filter rows based on column: 'bathroomcount'
    df_clean = df_clean[df_clean['bathroomcount'] <= 8]
    # Filter rows based on column: 'bedroomcount'
    df_clean = df_clean[df_clean['bedroomcount'] <= 6]
    # Filter rows based on column: 'livingarea'
    df_clean = df_clean[df_clean['livingarea'] < 350]
    # Filter rows based on column: 'numberoffacades'
    df_clean = df_clean[df_clean['numberoffacades'] <= 4]
    # Filter rows based on column: 'showercount'
    df_clean = df_clean[df_clean['showercount'] <= 3]
    # Filter rows based on column: 'surfaceofplot'
    df_clean = df_clean[df_clean['surfaceofplot'] <= 1000]
    # Filter rows based on column: 'toiletcount'
    df_clean = df_clean[df_clean['toiletcount'] <= 4]
    # Filter rows based on column: 'bathroomcount'
    df_clean = df_clean[df_clean['bathroomcount'] <= 3]
    return df_clean

df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

,bathroomcount,bedroomcount,district,floodingzone,garden,kitchen,livingarea,numberoffacades,peb,price,...,region,showercount,stateofbuilding,subtypeofproperty,surfaceofplot,swimmingpool,terrace,toiletcount,typeofproperty,typeofsale
0,1,1,Brugge,NON_FLOOD_ZONE,0,1,29.0,2,B,99000,...,Flanders,0,4,flat_studio,203,0,1,1,2,residential_sale
2,2,4,Brugge,NON_FLOOD_ZONE,0,1,111.0,2,B,399000,...,Flanders,0,4,house,0,0,0,2,1,residential_sale
3,1,4,Veurne,NON_FLOOD_ZONE,1,1,113.6,2,F,230000,...,Flanders,1,2,house,170,0,1,2,1,residential_sale
4,0,2,Hasselt,NON_FLOOD_ZONE,0,1,92.0,2,B,198000,...,Flanders,0,5,apartment,400,0,1,1,2,residential_sale
5,1,1,Brussels,NON_FLOOD_ZONE,0,3,50.0,2,E,215000,...,Brussels,1,5,apartment,143,0,1,1,2,residential_sale


In [31]:
print(df_clean_1.columns)


Index(['bathroomcount', 'bedroomcount', 'district', 'floodingzone', 'garden',
       'kitchen', 'livingarea', 'numberoffacades', 'peb', 'price', 'province',
       'region', 'showercount', 'stateofbuilding', 'subtypeofproperty',
       'surfaceofplot', 'swimmingpool', 'terrace', 'toiletcount',
       'typeofproperty', 'typeofsale'],
      dtype='object')


Categorize str values
constructionyear, district, floodingzone, subtypeofproperty, typeofsale

In [32]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['district', 'floodingzone', 'subtypeofproperty', 'typeofsale', 'peb','province', 'region']

data_to_encode = df_clean_1[columns_to_encode]

one = OneHotEncoder()

encoded_data = one.fit_transform(data_to_encode)

encoded_df = pd.DataFrame(encoded_data.toarray(), columns=one.get_feature_names_out(columns_to_encode))

df_final = pd.concat([df_clean_1.drop(columns=columns_to_encode), encoded_df], axis=1)

print(df_final.shape)
print(df_final.info())
df_final.to_csv('final_dataset.csv', index=False)
df_final.head()


(93332, 120)
<class 'pandas.core.frame.DataFrame'>
Index: 93332 entries, 0 to 81968
Columns: 120 entries, bathroomcount to region_Wallonie
dtypes: float64(120)
memory usage: 86.2 MB
None


,bathroomcount,bedroomcount,garden,kitchen,livingarea,numberoffacades,price,showercount,stateofbuilding,surfaceofplot,...,province_Hainaut,province_Limburg,province_Liège,province_Luxembourg,province_Namur,province_Walloon Brabant,province_West Flanders,region_Brussels,region_Flanders,region_Wallonie
0,1.0,1.0,0.0,1.0,29.0,2.0,99000.0,0.0,4.0,203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2.0,4.0,0.0,1.0,111.0,2.0,399000.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,4.0,1.0,1.0,113.6,2.0,230000.0,1.0,2.0,170.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,2.0,0.0,1.0,92.0,2.0,198000.0,0.0,5.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,1.0,0.0,3.0,50.0,2.0,215000.0,1.0,5.0,143.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Machine learning process

In [33]:
correlations = df_final.drop(columns=['price']).corrwith(df_final['price'])

print(correlations)



bathroomcount               0.259586
bedroomcount                0.375350
garden                      0.075140
kitchen                     0.314474
livingarea                  0.495621
                              ...   
province_Walloon Brabant    0.008028
province_West Flanders      0.005201
region_Brussels            -0.002545
region_Flanders             0.002449
region_Wallonie            -0.000996
Length: 119, dtype: float64


C:\Users\pieta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\pieta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [34]:
df_final

,bathroomcount,bedroomcount,garden,kitchen,livingarea,numberoffacades,price,showercount,stateofbuilding,surfaceofplot,...,province_Hainaut,province_Limburg,province_Liège,province_Luxembourg,province_Namur,province_Walloon Brabant,province_West Flanders,region_Brussels,region_Flanders,region_Wallonie
0,1.0,1.0,0.0,1.0,29.0,2.0,99000.0,0.0,4.0,203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2.0,4.0,0.0,1.0,111.0,2.0,399000.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,4.0,1.0,1.0,113.6,2.0,230000.0,1.0,2.0,170.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,2.0,0.0,1.0,92.0,2.0,198000.0,0.0,5.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,1.0,0.0,3.0,50.0,2.0,215000.0,1.0,5.0,143.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
81960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
81966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [35]:
from sklearn.model_selection import train_test_split

df = df_final

y = np.array(df ['price'])
X = np.array(df.drop(columns=['price']))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (74665, 119)
X_test shape: (18667, 119)
y_train shape: (74665,)
y_test shape: (18667,)


Cleaning X train, test


In [36]:
def clean_data(X_train_df):
    for col in X_train_df.columns:
        X_train_df[col] = X_train_df[col].fillna(X_train_df[col].median())
    return X_train_df

X_train_df = pd.DataFrame(X_train.tolist() if len(X_train.shape) > 2 else X_train)
X_train_df_clean = clean_data(X_train_df.copy())
print(X_train_df_clean.head())

   0    1    2    3      4    5    6    7      8    9    ...  109  110  111  \
0  1.0  1.0  1.0  0.0   61.0  2.0  1.0  4.0  651.0  0.0  ...  0.0  0.0  0.0   
1  2.0  4.0  0.0  1.0  192.0  2.0  1.0  4.0    0.0  0.0  ...  0.0  0.0  0.0   
2  1.0  4.0  0.0  2.0  193.0  4.0  1.0  4.0  379.0  0.0  ...  1.0  0.0  0.0   
3  1.0  2.0  0.0  1.0  108.0  3.0  0.0  2.0  536.0  0.0  ...  0.0  0.0  0.0   
4  1.0  3.0  0.0  2.0  126.0  3.0  1.0  3.0  400.0  0.0  ...  0.0  0.0  0.0   

   112  113  114  115  116  117  118  
0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  1.0  
3  0.0  0.0  0.0  1.0  0.0  1.0  0.0  
4  0.0  0.0  0.0  1.0  0.0  1.0  0.0  

[5 rows x 119 columns]


In [37]:
def clean_data(X_test_df):
    for col in X_test_df.columns:
        if X_test_df[col].isnull().any():
            X_test_df[col].fillna(X_test_df[col].median(), inplace=True)
    return X_test_df

X_test_df = pd.DataFrame(X_test.tolist() if len(X_test.shape) > 2 else X_test)
X_test_df_clean = clean_data(X_test_df.copy())
print(X_test_df_clean.head())




   0    1    2    3      4    5    6    7      8    9    ...  109  110  111  \
0  0.0  1.0  0.0  1.0  111.0  2.0  1.0  2.0   50.0  0.0  ...  0.0  1.0  0.0   
1  1.0  2.0  0.0  1.0   99.0  4.0  1.0  5.0  387.0  0.0  ...  0.0  0.0  0.0   
2  1.0  2.0  0.0  1.0  113.0  2.0  1.0  4.0  275.0  0.0  ...  0.0  0.0  0.0   
3  2.0  3.0  0.0  3.0  190.0  3.0  1.0  4.0  408.0  1.0  ...  0.0  1.0  0.0   
4  1.0  2.0  0.0  3.0   87.0  3.0  1.0  4.0  101.0  0.0  ...  0.0  0.0  0.0   

   112  113  114  115  116  117  118  
0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  1.0  0.0  

[5 rows x 119 columns]


C:\Users\pieta\AppData\Local\Temp\ipykernel_29588\2205198887.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_test_df[col].fillna(X_test_df[col].median(), inplace=True)
C:\Users\pieta\AppData\Local\Temp\ipykernel_29588\2205198887.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

Cleaning Y train and Y test

In [38]:
def clean_data(y_train_df):
    y_train_df = y_train_df.fillna({0: y_train_df[0].median()})
    return y_train_df

y_train_df = pd.DataFrame(y_train.tolist() if len(y_train.shape) > 2 else y_train)
y_train_df_clean = clean_data(y_train_df.copy())
y_train_df_clean.head()



,0
0,320000.0
1,475000.0
2,420000.0
3,239000.0
4,579000.0


In [39]:
def clean_data(y_test_df):
    y_test_df = y_test_df.fillna({0: 0})
    return y_test_df

y_test_df = pd.DataFrame(y_test.tolist() if len(y_test.shape) > 2 else y_test)
y_test_df_clean = clean_data(y_test_df.copy())
print(y_test_df_clean.shape)

(18667, 1)


Utiliser des transformers pour choisir quelle variable est plus intérressante pour le modèle
https://www.youtube.com/watch?v=T4nZDuakYlU&list=PLO_fdPEVlfKoHQ3Ua2NtDL4nmynQC8YiS&index=9

Model training


In [40]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_state=42)
model.fit(X_train_df_clean, y_train_df_clean)
y_pred = model.predict(X_test_df_clean)

Learning rate set to 0.080934
0:	learn: 118249.6138197	total: 7.45ms	remaining: 7.44s
1:	learn: 114823.8679065	total: 12.9ms	remaining: 6.43s
2:	learn: 111638.5560133	total: 18.7ms	remaining: 6.21s
3:	learn: 108832.9899257	total: 24.1ms	remaining: 6.01s
4:	learn: 106315.0177131	total: 29.3ms	remaining: 5.83s
5:	learn: 104105.6455903	total: 35ms	remaining: 5.8s
6:	learn: 102206.8619225	total: 40.6ms	remaining: 5.76s
7:	learn: 100429.8698299	total: 47.3ms	remaining: 5.86s
8:	learn: 98911.6690666	total: 52.9ms	remaining: 5.83s
9:	learn: 97496.7178495	total: 58.7ms	remaining: 5.81s
10:	learn: 96286.6496282	total: 64.4ms	remaining: 5.79s
11:	learn: 95151.3975234	total: 69.9ms	remaining: 5.75s
12:	learn: 94201.3498713	total: 75.8ms	remaining: 5.75s
13:	learn: 93343.3809493	total: 81.3ms	remaining: 5.72s
14:	learn: 92612.7485506	total: 86.9ms	remaining: 5.7s
15:	learn: 91854.7474948	total: 92.7ms	remaining: 5.7s
16:	learn: 91229.4738194	total: 98.8ms	remaining: 5.71s
17:	learn: 90624.6859681	

In [41]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_df_clean, y_pred)
mse = mean_squared_error(y_test_df_clean, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_df_clean, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2 Score: {r2}")

model.save_model('catboost_model')

MAE: 92439.3437624693
MSE: 17112296959.592594
RMSE: 130813.97845640425
R^2 Score: 0.333623719664766


In [42]:
df_final.head(1)


,bathroomcount,bedroomcount,garden,kitchen,livingarea,numberoffacades,price,showercount,stateofbuilding,surfaceofplot,...,province_Hainaut,province_Limburg,province_Liège,province_Luxembourg,province_Namur,province_Walloon Brabant,province_West Flanders,region_Brussels,region_Flanders,region_Wallonie
0,1.0,1.0,0.0,1.0,29.0,2.0,99000.0,0.0,4.0,203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [43]:
import streamlit as st

model = open('catboost_model', 'rb')

df = df_final

def main ():

    st.title("House Price Prediction")  
    st.sidebar.title("House Price Prediction")
    st.markdown("Welcome to House Price Prediction")

    

    # variables for the model
    user_input = {}


    if st.button("Predict"):
        model = CatBoostRegressor()
        model.load_model('catboost_model')
        prediction = model.predict(user_input_df)
        st.write(f"Predicted Price: {prediction}")

if __name__ == '__main__':  
    main()

            
            
            



    

In [44]:
'''from sklearn.model_selection import GridSearchCV, cross_val_score
from catboost import CatBoostRegressor
import pandas as pd

# Définir le modèle de base
model = CatBoostRegressor(random_state=42)

param_grid = {
    'iterations': [100, 300, 500, 700],
    'learning_rate': [0.001, 0.01, 0.05],
    'depth': [3, 5, 7, 10, 12],
    'l2_leaf_reg': [0.1, 1, 3, 5, 10]
}


# Configurer la recherche en grille
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Exécuter la recherche en grille
grid_search.fit(X_train_df_clean, y_train_df_clean)

# Afficher les meilleurs hyperparamètres
print("Best parameters found: ", grid_search.best_params_)

# Évaluer les performances du modèle optimisé
best_model = grid_search.best_estimator_
cv_scores = cross_val_score(best_model, X_train_df_clean, y_train_df_clean, cv=5, scoring='neg_mean_absolute_error')

# Afficher les scores de validation croisée
print("Cross-validated scores (MAE): ", -cv_scores)
print("Mean CV score (MAE): ", -cv_scores.mean())

# Faire des prédictions et évaluer le modèle sur les données de test
y_pred = best_model.predict(X_test_df_clean)

# Calculer les métriques de performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test_df_clean, y_pred)
mse = mean_squared_error(y_test_df_clean, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_df_clean, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2 Score: {r2}")'''




'from sklearn.model_selection import GridSearchCV, cross_val_score\nfrom catboost import CatBoostRegressor\nimport pandas as pd\n\n# Définir le modèle de base\nmodel = CatBoostRegressor(random_state=42)\n\nparam_grid = {\n    \'iterations\': [100, 300, 500, 700],\n    \'learning_rate\': [0.001, 0.01, 0.05],\n    \'depth\': [3, 5, 7, 10, 12],\n    \'l2_leaf_reg\': [0.1, 1, 3, 5, 10]\n}\n\n\n# Configurer la recherche en grille\ngrid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=\'neg_mean_absolute_error\', n_jobs=-1)\n\n# Exécuter la recherche en grille\ngrid_search.fit(X_train_df_clean, y_train_df_clean)\n\n# Afficher les meilleurs hyperparamètres\nprint("Best parameters found: ", grid_search.best_params_)\n\n# Évaluer les performances du modèle optimisé\nbest_model = grid_search.best_estimator_\ncv_scores = cross_val_score(best_model, X_train_df_clean, y_train_df_clean, cv=5, scoring=\'neg_mean_absolute_error\')\n\n# Afficher les scores de validation crois